<a href="https://colab.research.google.com/github/SASambath69/notebooks/blob/main/Tabula_Global_Top_100_Companies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objectif

Le présent notebook a été réalisé pour effectuer de la Data Visualisation à partir de données présentes dans un fichier au format pdf. L'objectif principal a été de découvrir la librairie tabula qui permet cette extraction des données à partir de fichiers pdf. <p />
Les visualisations porteront sur les 100 entreprises les mieux valorisées par la société PricewaterhouseCoopers en termes de capitalisations boursières. Ce classement a été publié en Juillet 2020 sur une valorisation des sociétés en Juin 2020, Mars 2020 et Décembre 2019. <p />
Je précise que mon travail porte sur des données qui ont été rendues publiques ([lien vers la page Web source](https://www.presseportal.ch/fr/pm/100008191/100852732)).

# Import et Retraitement des DataFrames

Quelques précisions sur le travail réalisé :
*    Sauvegarde du fichier pdf sur mon Google Drive
*    Connexion à mon Google Drive pour importer le fichier sur Colab
*    Retraitement et mise en forme des données importées (les données qui m'intéressent sont réparties sur 5 pages du fichier)
*    Création de pivot tables qui serviront de base aux visualisations

In [ ]:
# ------------------------------ Connexion à Google Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ------------------------------ Import des Librairies

!pip install tabula-py
import tabula
import pandas as pd
import plotly.express as px

# ------------------------------ Import du PDF et Retraitement en DataFrame

df_global = pd.DataFrame()

for i in range(11,16):

  df = tabula.read_pdf("/content/drive/My Drive/Colab Notebooks/Global Top 100 Companies by Market Capitalisation 2020 PwC.pdf", encoding='utf-8', pages=i)

  df[0].iloc[1,4] = df[0].iloc[0,4] + '_' + df[0].iloc[1,4]
  df[0].iloc[1,5] = df[0].iloc[0,4] + '_' + df[0].iloc[1,5]
  df[0].iloc[1,6] = df[0].iloc[0,4] + '_' + df[0].iloc[1,6]
  df[0].iloc[1,7] = df[0].iloc[0,5] + '_' + df[0].iloc[1,7]
  df[0].iloc[1,8] = df[0].iloc[0,5] + '_' + df[0].iloc[1,8]
  df[0].iloc[1,9] = df[0].iloc[0,6] + '_' + df[0].iloc[1,9]
  df[0].iloc[1,10] = df[0].iloc[0,6] + '_' + df[0].iloc[1,10]

  for i in range(len(df[0].columns)):
    df[0].iloc[1,i] = df[0].iloc[1,i].replace('\r',' ')
    df[0].iloc[1,i] = df[0].iloc[1,i].replace(' ','_')

  df[0].columns = list(df[0].iloc[1,:])
  df[0] = df[0].drop(index=[0,1])
  df_global = pd.concat([df_global,df[0]], axis=0)

df_global['30_June_2020_Market_capitalisation_($bn)'] = df_global['30_June_2020_Market_capitalisation_($bn)'].apply(lambda x: x.replace(',',''))
df_global['30_June_2020_Market_capitalisation_($bn)'] = df_global['30_June_2020_Market_capitalisation_($bn)'].astype('int')
df_global['31_March_2020_Market_capitalisation_($bn)'] = df_global['31_March_2020_Market_capitalisation_($bn)'].apply(lambda x: x.replace(',',''))
df_global['31_March_2020_Market_capitalisation_($bn)'] = df_global['31_March_2020_Market_capitalisation_($bn)'].astype('int')

df_global = df_global.reset_index(drop=True)

# ------------------------------ Création de Pivot Tables

# Capitalisation par secteur
pivot_sector = pd.pivot_table(df_global, values=['30_June_2020_Market_capitalisation_($bn)','31_March_2020_Market_capitalisation_($bn)'], index='Sector', aggfunc='sum').sort_values(by='30_June_2020_Market_capitalisation_($bn)', ascending=False).reset_index()
pivot_sector = pd.pivot_table(df_global, values=['30_June_2020_Market_capitalisation_($bn)','31_March_2020_Market_capitalisation_($bn)'], index='Sector', aggfunc='sum').sort_values(by='30_June_2020_Market_capitalisation_($bn)', ascending=False).reset_index()
pivot_sector['Variation'] = pivot_sector['30_June_2020_Market_capitalisation_($bn)'] - pivot_sector['31_March_2020_Market_capitalisation_($bn)']
pivot_sector['%'] = round(pivot_sector['Variation'] / pivot_sector['31_March_2020_Market_capitalisation_($bn)'],2)
evolution_sector = []
pivot_sector['Variation'].apply(lambda x: evolution_sector.append('Increase' if x > 0 else 'Decrease'))
pivot_sector['Evolution'] = evolution_sector

# Capitalisation par localisation
pivot_location = pd.pivot_table(df_global, values=['30_June_2020_Market_capitalisation_($bn)','31_March_2020_Market_capitalisation_($bn)'], index='Location', aggfunc='sum').sort_values(by='30_June_2020_Market_capitalisation_($bn)', ascending=False).reset_index()
pivot_location['Variation'] = pivot_location['30_June_2020_Market_capitalisation_($bn)'] - pivot_location['31_March_2020_Market_capitalisation_($bn)']
pivot_location['%'] = round(pivot_location['Variation'] / pivot_location['31_March_2020_Market_capitalisation_($bn)'],2)
evolution_location = []
pivot_location['Variation'].apply(lambda x: evolution_location.append('Increase' if x > 0 else 'Decrease'))
pivot_location['Evolution'] = evolution_location

In [ ]:
# Aperçu des tables
df_global.head()

,Rank,Company_name,Location,Sector,30_June_2020_Rank_+/-_(VS_Mar_2020),30_June_2020_Rank_+/-_(VS_Dec_2019),30_June_2020_Market_capitalisation_($bn),31_March_2020_Rank,31_March_2020_Market_capitalisation_($bn),31_Dec_2019_Rank,31_Dec_2019_Market_capitalisation_($bn)
0,1,SAUDI ARABIAN OI,Saudi Arabia,Oil & Gas,0,0,1741,1,1602,1,"1,879"
1,2,APPLE INC,United States,Technology,1,0,1568,3,1113,2,"1,305"
2,3,MICROSOFT CORP,United States,Technology,-1,0,1505,2,1200,3,"1,203"
3,4,AMAZON.COM INC,United States,Consumer Services,0,1,1337,4,971,5,916
4,5,ALPHABET INC-A,United States,Technology,0,-1,953,5,799,4,923


In [ ]:
pivot_sector.head()

,Sector,30_June_2020_Market_capitalisation_($bn),31_March_2020_Market_capitalisation_($bn),Variation,%,Evolution
0,Technology,8139,6384,1755,0.27,Increase
1,Consumer Services,3815,3054,761,0.25,Increase
2,Health Care,3188,2874,314,0.11,Increase
3,Financials,3067,2929,138,0.05,Increase
4,Oil & Gas,2465,2241,224,0.10,Increase


In [ ]:
pivot_location.head()

,Location,30_June_2020_Market_capitalisation_($bn),31_March_2020_Market_capitalisation_($bn),Variation,%,Evolution
0,United States,15948,13203,2745,0.21,Increase
1,Mainland China,2963,2620,343,0.13,Increase
2,Saudi Arabia,1741,1602,139,0.09,Increase
3,Switzerland,841,795,46,0.06,Increase
4,France,528,444,84,0.19,Increase


# Analyse par Secteur d'Activité

*    Quels sont les secteurs d'activité les plus présents ?
*    Que représente chaque activité en termes de capitalisation boursière au totale ?

In [ ]:
fig = px.histogram(data_frame = df_global,
             x = 'Sector').update_xaxes(categoryorder = 'total descending')
fig.update_layout(title = "Répartition des sociétés par secteur d'activité", title_x = 0.5)
fig.show()

In [ ]:
fig = px.bar(data_frame = pivot_sector,
       x = 'Sector',
       y = '30_June_2020_Market_capitalisation_($bn)',
       color = 'Evolution')
fig.update_layout(title = "Capitalisation boursière totale par secteur d'activité et évolution par rapport à Mars 2020", title_x = 0.5)
fig.show()

# Analyse par Localisation

*    Dans quels pays se situent majoritairement ces entreprises ?
*    Qu'est-ce que cela représente en termes de capitalisation boursière totale ?

In [ ]:
fig = px.histogram(data_frame = df_global,
             x = 'Location').update_xaxes(categoryorder = 'total descending')
fig.update_layout(title = "Répartition des sociétés par pays", title_x = 0.5)
fig.show()

In [ ]:
# Quelles sont les sociétés côtées en France ?
df_global.loc[df_global['Location'] == 'France']

,Rank,Company_name,Location,Sector,30_June_2020_Rank_+/-_(VS_Mar_2020),30_June_2020_Rank_+/-_(VS_Dec_2019),30_June_2020_Market_capitalisation_($bn),31_March_2020_Rank,31_March_2020_Market_capitalisation_($bn),31_Dec_2019_Rank,31_Dec_2019_Market_capitalisation_($bn)
26,27,LVMH MOET HENNE,France,Consumer Goods,5,4,220,32,188,31,235
44,45,L'OREAL,France,Consumer Goods,2,5,179,47,146,50,165
71,72,SANOFI,France,Health Care,4,11,129,76,110,83,126


In [ ]:
fig = px.bar(data_frame = pivot_location,
       x = 'Location',
       y = '30_June_2020_Market_capitalisation_($bn)',
       color = 'Evolution')
fig.update_layout(title = "Capitalisation boursière totale par pays et évolution par rapport à Mars 2020", title_x = 0.5)
fig.show()